In [1]:
import geopandas as gpd
from osgeo import ogr
import pandas as pd
import tifffile as tiff #needed for the tif data for perry county
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix
from math import sin, cos, sqrt, atan2, radians
import sys
from shapely.geometry import Polygon, box, Point, LineString, MultiLineString
import pickle
import gurobipy as gp
from gurobipy import GRB
import tifffile as tiff #needed for the tif data for perry county
import xlwt
from xlwt import Workbook
from shapely.ops import snap, split, nearest_points
#from shapely.geometry import MultiPoint, LineString
#from dbfread import DBF
import osmnx as ox
import networkx as nx
import math
import gurobipy as gp
from gurobipy import GRB
import os
import csv
from sklearn.cluster import AgglomerativeClustering
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

In [2]:
# calculate distance
def haversinedist(lat1, lon1, lat2, lon2):
    R = 6373.0
    
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    return distance #unit in km

In [3]:
# read data
def readClusterFile(fileID):
    file = np.genfromtxt(fileID, delimiter=",", skip_header = 1)
    file = file[:,1:]
    return file
clusterfile = 'Centralized_elevcluster' + str(1) + '.csv'

In [4]:
os.chdir("/Users/yuelanzhu/Downloads/Research/my code file")
# read data
def readClusterFile(fileID):
    file = np.genfromtxt(fileID, delimiter=",", skip_header = 1)
    file = file[:,1:]
    return file
clusterfile = 'Centralized_elevcluster' + str(1) + '.csv'
building_coords = readClusterFile(clusterfile)

In [5]:
# pipe parameters
pipesize = [0.05, 0.06, 0.08, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6] #m
pipesize_str, pipecost = gp.multidict({'0.05': 8.7, '0.06': 9.5, '0.08': 11, \
                                                   '0.1': 12.6, '0.15': 43.5,'0.2': 141, '0.25': 151, '0.3': 161, '0.35':230, '0.4': 246, '0.45':262, 
                                                   '0.5':292, '0.6':315}) #$/m
excavation = 25
bedding_cost_sq_ft = 6
capital_cost_pump_station = 166000
ps_flow_cost = 0.19
ps_OM_cost = 175950

In [6]:
# creat MST

graph = []
mstree = []
def createMSTnx(dataframe,n):
    #only takes longitude and latitude from the cluster data, note we also have elevation and stuff there too
    #coordinates = twoDcluster
    cluster = AgglomerativeClustering(n_clusters=n, affinity='euclidean', linkage='ward')
    cluster.fit_predict(building_coords[:,1:2])
    # add cluster to dataframe
    dataframe['cluster.labels_'] = cluster.labels_.tolist()
    for k in range(n):
        cluster_label = dataframe[dataframe['cluster.labels_'] == k]
        cluster_label = cluster_label[['longitude','latitude']]
        latlon = cluster_label.to_numpy()
        nrows, ncols = latlon.shape
        #creates graph
        graph.append(nx.Graph())
        weights = []
        #distance in km as weight between each point in the graph
        for i in range(nrows):
            graph[k].add_node(i,pos=(latlon[i,0],latlon[i,1]))
            for j in range(i+1,nrows):
                dist = haversinedist(latlon[i,1], latlon[i,0], latlon[j,1], latlon[j,0])
                weights.append(dist)
                graph[k].add_edge(i,j, weight = dist)
        #creates MST
        mstree.append(nx.minimum_spanning_tree(graph[k]))
    return mstree, graph

In [7]:
def result(df, num_c, cluster_labels, pipesize, pipecost, water_demand, HLmax, excavation, bedding_cost_sq_ft, capital_cost_pump_station, ps_OM_cost):

    # select highest and lowest nodes in cluster
    tank = df.loc[df.groupby(['cluster.labels_'])['elevation'].idxmax()].reset_index(drop=True)
    trem = df.loc[df.groupby(['cluster.labels_'])['elevation'].idxmin()].reset_index(drop=True)
                      
     # find label of tank node in cluster
    tank_lon = str(tank[tank['cluster.labels_'] == cluster_labels][['longitude']].iat[0,0])
    tank_lat = str(tank[tank['cluster.labels_'] == cluster_labels][['latitude']].iat[0,0])
    global tank_node
    for i in mstree[cluster_labels].nodes():
        if str(mstree[cluster_labels].nodes[i]['pos'][0]) == tank_lon and str(mstree[cluster_labels].nodes[i]['pos'][1]) == tank_lat:
            tank_node = i                  
    #create flow direction dict
    links = list(mstree[cluster_labels].edges)
    flow_dir = {}
    for i,j in links:
        if len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=i)) < len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=j)):
        # if node i is nearer to tank
           flow_dir[i,j]=(i,j)
        if len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=i)) > len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=j)):
         # if node j is nearer to tank
           flow_dir[i,j]=(j,i) 
    # find nodes for inflow and outflow
    innode = {}
    outnode = {}
    for i,j in links:
        if len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=i)) > len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=j)):
           # if node j is nearer to tank
            innode.setdefault(i, [])
            innode[i].append(j)
            outnode.setdefault(j, [])
            outnode[j].append(i)
        else:
           # if node i is nearer to tank
            innode.setdefault(j, [])
            innode[j].append(i)
            outnode.setdefault(i, [])
            outnode[i].append(j)
    for i in mstree[cluster_labels].nodes():
        innode.setdefault(i,[])
        outnode.setdefault(i, [])
   # directed graph
    mstree_dir = nx.DiGraph()
    mstree_dir.add_nodes_from(mstree[cluster_labels])
    mstree_dir.add_edges_from(flow_dir.values())
    weights = {}
    for i in list(mstree_dir.edges()):
        weights[i] = None
    for i,j in flow_dir:
        if flow_dir[i,j]==(i,j):
            weights[i,j]=mstree[cluster_labels].edges[i,j]['weight']
        if flow_dir[i,j]==(j,i):
            weights[j,i]=mstree[cluster_labels].edges[i,j]['weight']
    nx.set_edge_attributes(mstree_dir, values = 1, name = 'weight')
    nx.set_edge_attributes(mstree_dir, values = weights, name = 'weight')
    # determine iterate order
    walk = {}
    for i,j in mstree_dir.edges():
        walk[i,j] = len(nx.shortest_path(mstree_dir ,source=tank_node, target=j))
    sorted_walk = dict(sorted(walk.items(), key=lambda item: item[1], reverse = True))
    # pipe flow
    pipe_flow = {}
    for i,j in sorted_walk:
        if len(outnode[j]) == 0:
            pipe_flow[i,j] = 1.095E-5
        else:
            pipe_flow[i,j] = sum(pipe_flow[m,n] for m,n in sorted_walk if m == j) + 1.095E-5
    # head loss at largest dia        
    head_loss = {}
    for i in list(mstree_dir.edges()):
        head_loss[i] = None
    for i,j in list(mstree_dir.edges()):
        head_loss[i,j] = 10700*(pipe_flow[i,j]/140)**1.852*0.6*1000*mstree_dir[i][j]["weight"]
    # flow, distance and head loss for each edge
    edge_list = {}
    for i,j in links:
        if (i,j) == flow_dir[i,j]:
            edge_list[i,j] =[pipe_flow[i,j],mstree[cluster_labels][i][j]["weight"], head_loss[i,j]]
        else:
            edge_list[j,i] =[pipe_flow[j,i],mstree[cluster_labels][i][j]["weight"], head_loss[j,i]]
   #node elevation excavation in meters
   #upper bound is arbritrary maximum depth assuming 1 foot or 0.3048 meters of cover beneath the surface is needed for the pipes
   #a lower bound variable is created but not used. In future models might need to implement that depending on the site (digging too deep for excavation is not feasible for many projects)
    cluster = df[df['cluster.labels_'] == cluster_labels]
    cluster_elv = cluster[['elevation']]
    elevation_ub = dict()
    elevation_lb = dict()
    for i in range(len(cluster_elv)):
        GE=float(str(cluster_elv.values.tolist()[i])[1:-1])
        elevation_ub[i] = GE - 0.3
        elevation_lb[i] = GE - 30 #10
    GE=[]
    for i in range(len(cluster_elv)):
        GE.append(float(str(cluster_elv.values.tolist()[i])[1:-1]))
    #for i in range(mstree[cluster_labels].number_of_nodes()):
        #elevation_ub[i] = cluster_elv.iloc[[0]] - 0.3048
        #elevation_lb[i] = cluster_elv.iloc[[0]] - 30
        
    m = gp.Model('pipe and pump distribution')
    m.Params.timeLimit = 12000
   # binary variable indicating if at link ij pipe of diameter k is implemented
    d = m.addVars(mstree_dir.edges(), pipesize, vtype = GRB.BINARY, name = "diameter")
   #continuous variable representing the hydro loss in link ij.
    HL = m.addVars(mstree_dir.edges(),lb = 0, vtype = GRB.CONTINUOUS, name = "hydro loss")
    # hydro gradient
    #J = m.addVars(mstree[cluster_labels].edges(),lb = 0, vtype = GRB.CONTINUOUS, name = "hydro gradient")
   # binary variable indicating if at link ij the pump of type m is implemented.
    p = m.addVars(mstree_dir.edges(), vtype = GRB.BINARY, name = "PUMP")
   #continuous variable representing the pressure injected by the pump at link ij.
    pr = m.addVars(mstree_dir.edges(),lb = 0, vtype = GRB.CONTINUOUS, name = "PRESSURE BY PUMP")
   #pipe elevations at node i 
    e = m.addVars(mstree_dir.nodes(), lb = 0, vtype = GRB.CONTINUOUS, name = 'In Node Elevation')
   #continuous variable representing the head pressure at node i
    H = m.addVars(mstree_dir.nodes(),lb = 0, vtype = GRB.CONTINUOUS, name = "HEAD PRESSURE")
    for i,j in mstree_dir.edges():
#Hazen Williams equation, A = 10700, C=140, P472
        #m.addConstr(J[i,j] == 
                #10700*(edge_list[i,j][0]/140)**1.852*gp.quicksum(d[i,j,k]/(k**4.8704) for k in pipesize)
               #, "Hydro loss")
        m.addConstr(HL[i,j] == 10700*(edge_list[i,j][0]/140)**1.852*gp.quicksum(d[i,j,k]/(k**4.8704) for k in pipesize)*1000*mstree_dir[i][j]["weight"])
        m.addConstr(HL[i,j] <= HLmax)
#only one pipe per edge
        m.addConstr((d.sum(i, j, '*') == 1.0), "single size chosen") 
    
#only one pump per edge
        m.addConstr((p.sum(i, j, '*') <= 1.0), "single type chosen")
    
#injected pressure pr[i,j] must be less than a huge number.
        #m.addConstr(pr[i,j]<=p[i,j]*10**6)
    
    # injected pressure pr[i,j] must be less than the capacity of pump.
        m.addConstr(pr[i,j] <= p[i,j]*100, "pressure by pump less than 100 m ")
    # energy requirement
        m.addConstr(H[j] == H[i] - HL[i,j] + e[i] - e[j] + pr[i,j], "Energy balance")
    
        #if len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=i)) < len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=j)):
        # if node i is nearer to tank        
            #m.addConstr(H[j] == H[i] -HL[i,j] + e[i] - e[j] + pr[i,j], "Energy balance")
        #if len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=i)) > len(nx.shortest_path(mstree[cluster_labels],source=tank_node, target=j)):
        # if node j is nearer to tank        
            #m.addConstr(H[i] == H[j] -HL[i,j] + e[j] - e[i] + pr[i,j], "Energy balance")
# Velocity must be between 0.6 and 3 m/s to 30
        m.addConstr((
           edge_list[i,j][0] <= ((3.14/4)*gp.quicksum(d[i,j,k]*k**2 for k in pipesize)) * 3), "Velocity Max Constr"+ str([i,j]))
        #m.addConstr((
           #edge_list[i,j][0] >= ((3.14/4)*gp.quicksum(d[i,j,k]*k**2 for k in pipesize)) * 0.6), "Velocity Min Constr" + str([i,j]))  
    for i in range(len(cluster_elv)):
#pipe elevation must be betwenn lb and ub
        m.addConstr(e[i] >= elevation_lb[i], "PIPE ELV LB")
        m.addConstr(e[i] <= elevation_ub[i], "PIPE ELV UB")
    
        m.addConstr(H[i] >= 31.646232670592248, "Min head pressure") #45 psi, covert it to m
        m.addConstr(H[i] <= 56.26, "Max head pressure") #80 psi, covert it to m   
        
        
    #m.addConstr(H[tank_node] == 31.646232670592248)                
                      
    # pipe cost 
    obj1 = gp.quicksum(1000 * mstree_dir[i][j]["weight"] * gp.quicksum(pipecost[str(k)] * d[i, j, k] 
                                                           for k in pipesize) for i,j in mstree_dir.edges())
    
    # Capital cost of pump stations
    obj2 = gp.quicksum(p.sum(i, j) * (capital_cost_pump_station + ps_flow_cost*edge_list[i,j][0]) for i,j in mstree_dir.edges())
    #Operation and maintenance costs
    obj3 = gp.quicksum(p.sum(i, j)*ps_OM_cost for i, j in mstree_dir.edges())
    # Excavation cost
    obj4 = gp.quicksum((1 + gp.quicksum(d[i,j,k]*k for k in pipesize)) *1000* mstree_dir[i][j]["weight"] * bedding_cost_sq_ft + excavation * \
                               (1 + gp.quicksum(d[i,j,k]*k for k in pipesize)) *1000* mstree_dir[i][j]["weight"] * 0.5 * ((GE[i] - e[i]) + (GE[j] - e[j])) for i, j in mstree_dir.edges())

    # Capital cost of pump stations
    #obj2 = gp.quicksum(1000 * mstree_dir[i][j]["weight"] * gp.quicksum(pumpcost[pumptype.index(m)] * p[i, j, m] 
                                                           #for m in pumptype) for i,j in mstree_dir.edges()) #+penalty

    #Operation and maintenance costs
    #obj3 = gp.quicksum(p.sum(i, j)*ps_OM_cost for i, j in mstree_dir.edges())

    obj = obj1+obj2+obj3+obj4                  
    m.setObjective(obj, GRB.MINIMIZE)
    m.optimize()
    
    
    # diameter
    diameter = { k : v.X for k,v in d.items()}
    diameter = { k : v.X for k,v in d.items() if v.X ==1}
    diameter = list(diameter.keys())
    d = []
    for i in range(len(diameter)):
        d.append(diameter[i][2])
        
    # pump
    pump = { k : v.X for k,v in p.items() if v.X ==1}
    pump = list(pump.keys())
    p = [x[0] for x in pump]
    color_map = []
    for node in mstree[cluster_labels]:
        if node in p:
            color_map.append('blue')
        else: 
            color_map.append('c')
        if node == tank_node:
            color_map[node]=('red') 

            
    #Hydro loss
    hydro_loss = { k : v.X for k,v in HL.items()}

    
    #elev
    ele = { k : v.X for k,v in e.items()}
    
    #head pressure
    h = { k : v.X for k,v in H.items()}
        
    
    return d, color_map, diameter, edge_list, hydro_loss, ele, h
                      

In [8]:
#convert data to dataframe
df = pd.DataFrame(building_coords,
                  columns = ['longitude','latitude','elevation'])
g_mst = createMSTnx(df,1)
dia = []
pum = []
for i in range(0,1):
    output = result(df, 1, i, pipesize, pipecost, 1.095E-5, 1e+5, excavation, bedding_cost_sq_ft, capital_cost_pump_station, ps_OM_cost)
    dia.append(output[0])
    pum.append(output[1])

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-14
Set parameter TimeLimit to value 12000
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 12984 rows, 21242 columns and 63724 nonzeros
Model fingerprint: 0x72ecb725
Model has 30680 quadratic objective terms
Variable types: 4722 continuous, 16520 integer (16520 binary)
Coefficient statistics:
  Matrix range     [8e-08, 2e+05]
  Objective range  [1e+02, 1e+06]
  QObjective range [1e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-05, 1e+05]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 9444 rows and 1873 columns
Presolve time: 0.08s
Presolved: 32834 rows, 48663 columns, 125166 nonzeros
Variable types: 32836 continuous, 15827 integer (15827 binary)
Found heuristic solution: objective 2.794975

 44673 10972 infeasible  124      3716892.94 3715207.55  0.05%   5.2  121s
H46674 11439                    3716869.4369 3715225.54  0.04%   5.1  123s
 46995 12311 3715623.58   88  129 3716869.44 3715226.35  0.04%   5.1  125s
 49928 14986 3716614.71  108  119 3716869.44 3715246.15  0.04%   5.0  130s
*50853 14913             320    3716864.8687 3715248.17  0.04%   5.0  130s
 54596 18369 3716588.86  123  109 3716864.87 3715268.81  0.04%   4.9  136s
 56628 19697 3716813.56  171   72 3716864.87 3715274.40  0.04%   4.8  140s
*57774 19593             310    3716859.4922 3715283.12  0.04%   4.8  140s
H59433 18098                    3716749.7384 3715290.46  0.04%   4.8  144s
H60050 16930                    3716714.6986 3715296.27  0.04%   4.8  168s
 60053 16919 3715430.96   71  129 3716714.70 3715296.27  0.04%   4.8  170s
 60085 16941 3716627.44  211  354 3716714.70 3715296.27  0.04%   4.8  175s
 60110 16957 3715906.20   90  441 3716714.70 3715296.27  0.04%   4.8  180s
 60150 16984 3716329.12  

 218125 83898 3716460.47  142  127 3716607.02 3715796.98  0.02%   6.9  617s
 219773 84558 3716159.42  136  139 3716607.02 3715798.19  0.02%   6.9  620s
 222360 85610 3716502.28  151  125 3716607.02 3715799.77  0.02%   6.9  625s
 226367 87084     cutoff  117      3716607.02 3715802.57  0.02%   6.9  632s
 228921 88280 3716033.86  140  131 3716607.02 3715804.68  0.02%   6.9  638s
 230887 88667     cutoff  142      3716607.02 3715805.67  0.02%   6.9  640s
 233480 89649 3716531.28  155  113 3716607.02 3715807.35  0.02%   6.9  645s
 235659 90726 3715964.48  122  133 3716607.02 3715809.20  0.02%   7.0  650s
 238351 91846 3716194.03  129  129 3716607.02 3715810.87  0.02%   7.0  655s
 240883 92978 3716261.26  133  121 3716607.02 3715812.55  0.02%   7.0  660s
 244146 94035 3715914.31  116  137 3716607.02 3715814.42  0.02%   7.0  665s
 247685 95741 infeasible  143      3716607.02 3715816.92  0.02%   7.0  672s
 250541 96684 3716111.66  126  133 3716607.02 3715818.25  0.02%   7.0  676s
 253039 9790

 494413 179916 3716384.08  150  111 3716606.74 3715933.41  0.02%   7.5 1142s
 496801 180513 3716079.44  122  135 3716606.74 3715934.37  0.02%   7.5 1146s
 499159 181094     cutoff  154      3716606.74 3715935.29  0.02%   7.5 1152s
 499809 181585 3716096.43  131  141 3716606.74 3715935.35  0.02%   7.5 1155s
 503247 182446 3716124.80  143  129 3716606.74 3715936.65  0.02%   7.5 1161s
 506235 183304 3716046.85  121  137 3716606.74 3715937.63  0.02%   7.5 1166s
 508848 184131 3716518.14  144  125 3716606.74 3715938.67  0.02%   7.5 1172s
 510504 184506 3716388.47  157  117 3716606.74 3715939.21  0.02%   7.5 1175s
 514208 185583 3716136.80  138  145 3716606.74 3715940.59  0.02%   7.5 1182s
 516333 186435 3716326.96  129  141 3716606.74 3715941.46  0.02%   7.5 1187s
 517968 186883     cutoff  140      3716606.74 3715942.08  0.02%   7.5 1190s
 520524 187655 3716368.09  120  143 3716606.74 3715943.00  0.02%   7.5 1195s
 524558 188664 3716531.03  123  137 3716606.74 3715944.42  0.02%   7.6 1202s

 760343 250287 3716579.19  148  108 3716606.74 3716021.22  0.02%   7.8 1686s
 762941 251043 3716483.72  132  131 3716606.74 3716021.89  0.02%   7.8 1691s
 765568 251795     cutoff  129      3716606.74 3716022.82  0.02%   7.8 1696s
 768107 252530     cutoff  130      3716606.74 3716023.52  0.02%   7.8 1701s
 771181 253321 3716442.61  131  135 3716606.74 3716024.37  0.02%   7.8 1707s
 774261 253742 3716364.06  128  137 3716606.74 3716025.16  0.02%   7.8 1711s
 776454 254363 3716104.86  119  139 3716606.74 3716025.68  0.02%   7.8 1716s
 779100 255036     cutoff  126      3716606.74 3716026.47  0.02%   7.8 1721s
 781751 255538     cutoff  134      3716606.74 3716027.23  0.02%   7.8 1726s
 784607 256379 3716245.53  133  139 3716606.74 3716027.92  0.02%   7.8 1731s
 787263 257050 3716501.01  122  135 3716606.74 3716028.60  0.02%   7.8 1736s
 789748 257734 3716511.71  144  139 3716606.74 3716029.11  0.02%   7.8 1741s
 792360 258395 3716504.66  150  115 3716606.74 3716029.85  0.02%   7.8 1746s

 1027162 314863     cutoff  128      3716606.74 3716085.08  0.01%   8.0 2220s
 1029733 315587 3716481.40  122  143 3716606.74 3716085.51  0.01%   8.0 2226s
 1032618 316201 3716157.98  129  139 3716606.74 3716086.09  0.01%   8.0 2231s
 1035339 316800     cutoff  120      3716606.74 3716086.63  0.01%   8.0 2237s
 1038060 317283 3716387.26  131  144 3716606.74 3716087.31  0.01%   8.0 2242s
 1040477 317787 3716593.16  146  131 3716606.74 3716087.85  0.01%   8.0 2247s
 1042949 318299 3716397.12  119  155 3716606.74 3716088.42  0.01%   8.0 2252s
 1045342 318717 3716183.37  129  131 3716606.74 3716088.89  0.01%   8.0 2257s
 1046908 318891 3716508.34  136  137 3716606.74 3716089.15  0.01%   8.0 2260s
 1049062 319416 3716569.25  141  137 3716606.74 3716089.60  0.01%   8.0 2265s
 1052247 320048 infeasible  133      3716606.74 3716090.41  0.01%   8.0 2270s
 1054775 320535 3716498.00  141  117 3716606.74 3716090.96  0.01%   8.0 2275s
 1056724 321120 3716592.12  144  114 3716606.74 3716091.37  0.01

 1284178 370342 3716211.97  129  159 3716606.74 3716131.87  0.01%   8.2 2756s
 1287160 370969 3716174.84  120  151 3716606.74 3716132.29  0.01%   8.2 2761s
 1289691 371440 3716253.83  125  141 3716606.74 3716132.69  0.01%   8.2 2766s
 1292251 371864 3716205.20  129  138 3716606.74 3716133.26  0.01%   8.2 2771s
 1294861 372375     cutoff  132      3716606.74 3716133.65  0.01%   8.2 2777s
 1295936 372630 3716362.07  114  153 3716606.74 3716133.88  0.01%   8.2 2780s
 1298552 373241 3716554.23  122  134 3716606.74 3716134.38  0.01%   8.2 2786s
 1301520 373896 3716441.43  125  149 3716606.74 3716134.90  0.01%   8.2 2791s
 1304311 374442 3716272.03  135  141 3716606.74 3716135.30  0.01%   8.2 2796s
 1306659 375027     cutoff  143      3716606.74 3716135.67  0.01%   8.3 2801s
 1309723 375528     cutoff  121      3716606.74 3716136.11  0.01%   8.3 2806s
 1311155 376058 3716178.99  104  177 3716606.74 3716136.25  0.01%   8.3 2810s
 1313602 376525 3716374.88  133  149 3716606.74 3716136.66  0.01

 1539901 424664     cutoff  128      3716606.74 3716168.99  0.01%   8.4 3287s
 1543285 425138 3716271.59  121  153 3716606.74 3716169.45  0.01%   8.4 3292s
 1544558 425525 3716189.70  123  143 3716606.74 3716169.67  0.01%   8.4 3295s
 1547562 426180 3716581.81  142  138 3716606.74 3716170.07  0.01%   8.4 3301s
 1550760 426743 3716568.68  135  138 3716606.74 3716170.47  0.01%   8.4 3306s
 1553258 427390 3716598.26  152  129 3716606.74 3716170.72  0.01%   8.4 3311s
 1555899 427955 3716516.70  123  126 3716606.74 3716171.11  0.01%   8.4 3316s
 1558220 428462 3716522.20  130  141 3716606.74 3716171.40  0.01%   8.4 3320s
 1560654 428935 3716314.20  133  140 3716606.74 3716171.71  0.01%   8.4 3325s
 1563266 429467     cutoff  117      3716606.74 3716172.13  0.01%   8.4 3330s
 1564978 430003 3716554.69  153  135 3716606.74 3716172.32  0.01%   8.4 3335s
 1567733 430725 3716214.78  119  171 3716606.74 3716172.65  0.01%   8.4 3340s
 1570367 430973 3716452.62  128  139 3716606.74 3716172.91  0.01

 1806576 487404 3716324.62  109  149 3716606.74 3716199.80  0.01%   8.4 3827s
 1808223 487717 3716482.84  134  133 3716606.74 3716199.98  0.01%   8.4 3830s
 1811096 488307     cutoff  124      3716606.74 3716200.27  0.01%   8.4 3835s
 1813852 488847 3716344.63  140  125 3716606.74 3716200.62  0.01%   8.4 3840s
 1816705 489503 3716523.68  134  149 3716606.74 3716200.89  0.01%   8.4 3846s
 1819253 490214     cutoff  123      3716606.74 3716201.12  0.01%   8.4 3851s
 1821936 490832 3716356.54  132  139 3716606.74 3716201.45  0.01%   8.4 3857s
 1824028 491512 3716429.03  139  119 3716606.74 3716201.67  0.01%   8.4 3862s
 1826741 492268 3716419.18  143  127 3716606.74 3716202.03  0.01%   8.4 3867s
 1829028 492919 3716538.12  121  150 3716606.74 3716202.22  0.01%   8.4 3873s
 1830869 493403     cutoff  130      3716606.74 3716202.37  0.01%   8.4 3876s
 1832850 493802 3716389.00  155  113 3716606.74 3716202.65  0.01%   8.4 3880s
 1835559 494470 3716403.38  113  145 3716606.74 3716202.86  0.01

 2072451 551636 3716272.54  110  152 3716606.74 3716226.12  0.01%   8.3 4360s
 2075469 552306 3716265.57  141  141 3716606.74 3716226.35  0.01%   8.3 4365s
 2078245 553001 3716385.35  116  154 3716606.74 3716226.65  0.01%   8.3 4370s
 2080960 553849     cutoff  132      3716606.74 3716226.87  0.01%   8.3 4376s
 2083576 554542 3716390.34  124  147 3716606.74 3716227.11  0.01%   8.3 4381s
 2086215 555244 3716497.31  124  143 3716606.74 3716227.38  0.01%   8.3 4386s
 2088906 555938 3716538.94  123  151 3716606.74 3716227.61  0.01%   8.3 4392s
 2091613 556648 3716550.81  151  129 3716606.74 3716227.83  0.01%   8.3 4397s
 2094246 557138     cutoff  147      3716606.74 3716228.04  0.01%   8.3 4402s
 2096592 557796 3716517.11  118  133 3716606.74 3716228.25  0.01%   8.3 4407s
 2098256 558222 3716266.35  136  137 3716606.74 3716228.41  0.01%   8.3 4410s
 2100805 558898 3716291.78  127  129 3716606.74 3716228.63  0.01%   8.3 4415s
 2103690 559480 3716337.29  140  121 3716606.74 3716228.90  0.01

In [9]:
def d_joint(n):
    if n == 0:
        return dia[0]
    else:
        d_all = dia[n]+d_joint(n-1)
        return d_all

In [10]:
def p_joint(n):
    if n == 0:
        return pum[0]
    else:
        p_all = pum[n]+p_joint(n-1)
        return p_all

In [11]:
def g_joint(n):
    if n == 0:
        return g_mst[0][0]
    else:
        g = nx.disjoint_union(g_joint(n-1),g_mst[0][n])
        return g

In [ ]:
# edge legend
fig, ax = plt.subplots(figsize=(6, 1))
fig.subplots_adjust(bottom=0.5,left=1, right=6)
norm = mpl.colors.Normalize(vmin=5, vmax=60)

cb = mpl.colorbar.ColorbarBase(ax, cmap=plt.cm.Blues,
                                norm=norm,
                                orientation='horizontal')
cb.set_label('Pipesize[cm]',fontsize=20)

In [ ]:
diaedge = []
for i in range(len(output[2])):
    diaedge.append((output[2][i][0],output[2][i][1]))

In [ ]:
diff = list(set(list(output[3].keys()))-set(diaedge))

In [ ]:
diff

In [ ]:
miss1 = list(output[3].keys()).index(diff[0])

In [ ]:
miss2 = list(output[3].keys()).index(diff[1])

In [ ]:
miss2

In [ ]:
len(output[2])

In [ ]:
len(g_mst[0][0].edges())

In [ ]:
len(dia[0])

In [ ]:
d5 = result(df, 9, 5, pipesize, pipecost, 1.095E-5, 1e+5, excavation, bedding_cost_sq_ft, capital_cost_pump_station, ps_OM_cost)[2]

In [ ]:
dia[0].insert(miss1,0.2)
dia[0].insert(miss2,0.2)

In [ ]:
len(g_mst[0][8].edges())

In [ ]:
len(dia[8])

In [ ]:
d9=result(df, 10, 8, pipesize, pipecost, 1.095E-5, 1e+5, excavation, bedding_cost_sq_ft, capital_cost_pump_station, ps_OM_cost)[2]

In [ ]:
diaedge = []
for i in range(len(d9)):
    diaedge.append((min(d9[i][0],d9[i][1]),max(d9[i][0],d9[i][1])))
diff = list(set(list(g_mst[0][8].edges().keys()))-set(diaedge))

In [ ]:
list(g_mst[0][8].edges().keys())[54]

In [ ]:
list(g_mst[0][8].edges().keys())[56]

In [ ]:
diff

In [ ]:
miss1 = list(output[3].keys()).index(diff[0])
miss2 = list(output[3].keys()).index(diff[1])

In [ ]:
dia[8].insert(54,0.2)
dia[8].insert(56,0.2)

In [ ]:
# check
DiaColorMap = []
if len(g_joint(8).edges())==len(d_joint(8)):
    DiaColorMap = d_joint(8)
else:
    dlen = []
    dlen.append(len(dia[0]))
    for i in range(1,9):
        dlen.append(len(d_joint(i))-len(d_joint(i-1)))
    for i in range(0,9):
        if dlen[i]!= len(g_mst[0][i].edges()):
            print(i)
        else:
            print("everything is ok")

In [12]:
pos = {city:(long, lat) for (city, (lat,long)) in nx.get_node_attributes(g_joint(0), 'pos').items()}
plt.figure(3,figsize=(120,120)) 
nx.draw(g_joint(0), pos, with_labels=True, 
        node_size=600, node_color = p_joint(0), 
        edgelist=g_joint(0).edges(),edge_color = d_joint(0),
        edge_cmap=plt.cm.Blues, width=10)
# node legend
pump_patch = mlines.Line2D([], [], color='blue', marker='o', linestyle='None',
                          markersize=100, label='Pump')
tank_patch = mlines.Line2D([], [], color='red', marker='o', linestyle='None',
                          markersize=100, label='Tank')
nonpump_patch = mlines.Line2D([], [], color='c', marker='o', linestyle='None',
                          markersize=100, label='Non-pump')
plt.legend(handles=[pump_patch, tank_patch, nonpump_patch],fontsize=100)
#plt.title('The least cost system sizing for 1 subnework',fontsize=200)
plt.show()